# Playing Differentiated Demand Environemnt

## Imports and initialize


In [6]:
#Imports

from marketsai.markets.diff_demand import DiffDemand

#import ray

from ray import tune, shutdown, init
from ray.tune.registry import register_env
from ray.rllib.agents.a3c.a2c import A2CTrainer
from ray.rllib.agents.dqn.dqn import DQNTrainer
from ray.tune.integration.mlflow import MLflowLoggerCallback
from ray.rllib.utils.exploration.epsilon_greedy import EpsilonGreedy
from ray.rllib.utils.schedules.exponential_schedule import ExponentialSchedule

import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

In [7]:
# STEP 0: Inititialize ray
NUM_CPUS = 11
shutdown()
init(num_cpus=NUM_CPUS, 
    logging_level=logging.ERROR,
)

{'node_ip_address': '192.168.1.202',
 'raylet_ip_address': '192.168.1.202',
 'redis_address': '192.168.1.202:51686',
 'object_store_address': '/tmp/ray/session_2021-04-13_13-13-54_817863_90750/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-04-13_13-13-54_817863_90750/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-04-13_13-13-54_817863_90750',
 'metrics_export_port': 63596,
 'node_id': '46b1b40f925ca2c5f1cd3bbcd7163c2e9c7277a757c14602886498c7'}

In [8]:
# STEP 1: register environment
register_env("diffdemand", DiffDemand)
env = DiffDemand()
policy_ids = [f"policy_{i}" for i in range(env.n_agents)]

## Experminet Configuration

In [9]:
# STEP 2: Experiment configuration

#Experiment configuration
test=True
date="April13"
env_label="Diff_Dd"
if (test==True): 
    MAX_STEPS = 20 * 1000
    exp_label =env_label+"_test_"+date 
else: 
    MAX_STEPS = 3000 * 1000
    exp_label ="_run_"+date 

verbosity=2
stop = {"episodes_total": MAX_STEPS//100}

#Environment configuration
PRICE_BAND_WIDE = 0.1
LOWER_PRICE = 1.47 - PRICE_BAND_WIDE
HIGHER_PRICE = 1.93 + PRICE_BAND_WIDE
DEC_RATE = float(math.e ** (-4 * 10 ** (-6)))
DEC_RATE_HIGH = float(math.e ** (-4 * 10 ** (-6) * 4))

env_config = {
    "mkt_config": {
    "lower_price": [LOWER_PRICE for i in range(env.n_agents)],
    "higher_price": [HIGHER_PRICE for i in range(env.n_agents)],
    "parameteres": {
                "cost": [1 for i in range(env.n_agents)],
                "values": [2 for i in range(env.n_agents)],
                "ext_demand": 0,
                "substitution": 0.25,
    },
    "space_type": "MultiDiscrete",
    "gridpoints": 16,
    }
}

exploration_config = {"type": "EpsilonGreedy",
    "epsilon_schedule": ExponentialSchedule(
      schedule_timesteps = 1,
      framework="Torch",
      initial_p=1.0,
      decay_rate=DEC_RATE,
    ),
}

training_config = {
    "gamma": 0.95,
    "lr": 0.15,
    "env": "diffdemand",
    "exploration_config": exploration_config,
    "env_config": env_config,
    "horizon": 100,
    "soft_horizon": True,
    "no_done_at_end": True,
    "multiagent": {
        "policies": {
            policy_ids[i]: (
                None,
                env.observation_space["agent_{}".format(i)],
                env.action_space["agent_{}".format(i)],
                {},
            )
            for i in range(env.n_agents)
        },
        "policy_mapping_fn": (lambda agent_id: policy_ids[int(agent_id.split("_")[1])]),
    },
    "framework": "torch",
    "num_workers": NUM_CPUS - 1,
    "num_gpus": 0,
    "timesteps_per_iteration": 1000,
    "normalize_actions": False,
}


#stop = {"training_iteration": MAX_STEPS//1000}
#stop = {"info/num_steps_trained": MAX_STEPS}

# # Experiments

In [10]:
#DQN Methods: DQN, APEX, R2D2

# algo_list = ["DQN", "APEX", "R2D2"] 
algo_list=["DQN"]
for i in range(len(algo_list)):
    exp_name = exp_label+algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )


training_config_RBW=training_config.copy()
training_config_RBW["n_step"] = 5
training_config_RBW["noisy"] = True
training_config_RBW["num_atoms"] = 10
training_config_RBW["v_min"] = 0.5
training_config_RBW["v_min"] = 2


exp_name = exp_label+"RAINBOW"
results = tune.run(
    "DQN",
    name=exp_name,
    config=training_config_RBW,
    #checkpoint_freq=250,
    checkpoint_at_end=True,
    stop=stop,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
    verbose=verbosity
)


(raylet) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(raylet)   warnings.warn(msg)
(raylet) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(raylet)   warnings.warn(msg)


 been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92297) 2021-04-13 13:14:10,854	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92300) 2021-04-13 13:14:10,803	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92300) 2021-04-13 13:14:10,803	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92294) 2021-04-13 13:14:10,822	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92294) 2021-04-13 13:14:10,836	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Us

Trial DQN_diffdemand_a2222_00000 reported episode_reward_max=56.312076668098676,episode_reward_min=52.235194358618685,episode_reward_mean=54.20764147875052,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.452694502379327, 'policy_1': 25.06152365188502},policy_reward_max={'policy_0': 29.336373631598217, 'policy_1': 29.97234051715639},policy_reward_mean={'policy_0': 26.92677401243328, 'policy_1': 27.280867466317225},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.39579802147856197, 'mean_raw_obs_processing_ms': 0.6509710821864046, 'mean_inference_ms': 5.697864595626517, 'mean_action_processing_ms': 0.22757158510876074},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 22.433, 'learn_throughput': 1426.476, 'update_time_ms': 4.705},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(0.39403528, dtype=float32), 'cur_lr': 0.15, 'mean_q': 22.227054595947266, 'min_q': 21.524044036865234, 'max_q': 22.532318115234

Trial DQN_diffdemand_a2222_00000 reported episode_reward_max=56.312076668098676,episode_reward_min=50.599593098790365,episode_reward_mean=53.93104525920684,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.452694502379327, 'policy_1': 24.77796203157801},policy_reward_max={'policy_0': 29.336373631598217, 'policy_1': 29.97234051715639},policy_reward_mean={'policy_0': 26.84542977605708, 'policy_1': 27.085615483149763},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3838712816610335, 'mean_raw_obs_processing_ms': 0.6308215516362815, 'mean_inference_ms': 5.530091847271066, 'mean_action_processing_ms': 0.21993142842173924},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 21.403, 'learn_throughput': 1495.136, 'update_time_ms': 4.597},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(1.01224, dtype=float32), 'cur_lr': 0.15, 'mean_q': 18.138444900512695, 'min_q': 17.89937400817871, 'max_q': 18.42430305480957, '

Trial DQN_diffdemand_a2222_00000 reported episode_reward_max=57.19325147727085,episode_reward_min=50.599593098790365,episode_reward_mean=54.024033594532995,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.452694502379327, 'policy_1': 24.63238917933922},policy_reward_max={'policy_0': 29.525345758437457, 'policy_1': 29.97234051715639},policy_reward_mean={'policy_0': 26.953089808334738, 'policy_1': 27.070943786198246},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3748115926601789, 'mean_raw_obs_processing_ms': 0.616077705528509, 'mean_inference_ms': 5.400995083105828, 'mean_action_processing_ms': 0.21422875192593427},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 20.888, 'learn_throughput': 1531.958, 'update_time_ms': 4.446},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'grad_gnorm': array(1.4356979, dtype=float32), 'cur_lr': 0.15, 'mean_q': 13.957423210144043, 'min_q': 13.796136856079102, 'max_q': 14.21234226226806

...

cause its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=92388)   support_prob_per_action = nn.functional.softmax(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualen

 because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=92388)   support_prob_per_action = nn.functional.softmax(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtua

 because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=92388)   support_prob_per_action = nn.functional.softmax(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtua

ent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_policy.py:357: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=92388)   support_prob_per_action = nn.functional.softmax(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/rllib/agents/dqn/dqn_torch_model.py:145: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
(pid=92388)   z = torch.range(
(pid=92388) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_diffdemand_c272e_00000,TERMINATED,,20,54.3106,20000,52.3789,55.2743,49.7107,100


In [11]:
#Policy Gradient Methods: PG, A2C, A3C, PPO, APPO

# algo_list=["PG", "A2C", "A3C", "PPO", "APPO"]
algo_list=["PG", "PPO"]
for i in range(len(algo_list)):
    exp_name = exp_label + algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )


tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=92492) Instructions for updating:
(pid=92492) non-resource variables are not supported in the long term
(pid=92492) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=92492) Instructions for updating:
(pid=92492) non-resource variables are not supported in the long term
(pid=92493) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=92493) Instructions for updating:
(pid=92493) no

Trial PG_diffdemand_eaa26_00000 reported episode_reward_max=56.87811984250468,episode_reward_min=51.67661881624113,episode_reward_mean=54.17148503051676,episode_len_mean=100.0,episodes_this_iter=20,policy_reward_min={'policy_0': 24.215886870482993, 'policy_1': 25.128520037088308},policy_reward_max={'policy_0': 29.44089808723671, 'policy_1': 30.8025306474089},policy_reward_mean={'policy_0': 26.98991681673382, 'policy_1': 27.181568213782935},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.4453855757161696, 'mean_raw_obs_processing_ms': 0.5048339956004466, 'mean_inference_ms': 4.588108935994122, 'mean_action_processing_ms': 0.24583846360437978},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 1278.729, 'sample_throughput': 1564.053, 'learn_time_ms': 99.82, 'learn_throughput': 20036.114, 'update_time_ms': 6.388},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 71.7271728515625}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss': 70.18

Trial PG_diffdemand_eaa26_00000 reported episode_reward_max=56.274615806318764,episode_reward_min=51.78314819134301,episode_reward_mean=54.09769157270872,episode_len_mean=100.0,episodes_this_iter=20,policy_reward_min={'policy_0': 24.215886870482993, 'policy_1': 24.865572338239883},policy_reward_max={'policy_0': 29.314868967260576, 'policy_1': 30.8025306474089},policy_reward_mean={'policy_0': 27.085092779058723, 'policy_1': 27.01259879364999},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.47073790471799093, 'mean_raw_obs_processing_ms': 0.5384055599096483, 'mean_inference_ms': 4.875429438301872, 'mean_action_processing_ms': 0.25755846380460995},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 1337.472, 'sample_throughput': 1495.358, 'learn_time_ms': 106.131, 'learn_throughput': 18844.706, 'update_time_ms': 6.804},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 139.2841033935547}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss':

Trial PG_diffdemand_eaa26_00000 reported episode_reward_max=56.274615806318764,episode_reward_min=51.78314819134301,episode_reward_mean=54.14683575455469,episode_len_mean=100.0,episodes_this_iter=20,policy_reward_min={'policy_0': 24.752288223944895, 'policy_1': 24.214876028182054},policy_reward_max={'policy_0': 30.7863379281433, 'policy_1': 29.139651515694126},policy_reward_mean={'policy_0': 27.44229066420881, 'policy_1': 26.70454509034588},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.48731098535735595, 'mean_raw_obs_processing_ms': 0.562412989274758, 'mean_inference_ms': 5.0664008746617375, 'mean_action_processing_ms': 0.2664874402929105},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 1378.148, 'sample_throughput': 1451.223, 'learn_time_ms': 115.715, 'learn_throughput': 17283.819, 'update_time_ms': 6.526},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'policy_loss': 238.1383056640625}, 'policy_1': {'allreduce_latency': 0.0, 'policy_loss': 6

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PG_diffdemand_eaa26_00000,TERMINATED,,10,15.1349,20000,54.1468,56.2746,51.7831,100


...

Trial PPO_diffdemand_fea7d_00000 reported episode_reward_max=56.18908721681132,episode_reward_min=51.60864418261823,episode_reward_mean=54.118034302986,episode_len_mean=100.0,episodes_this_iter=40,policy_reward_min={'policy_0': 24.743845635805606, 'policy_1': 24.568367919442},policy_reward_max={'policy_0': 29.063576027136055, 'policy_1': 28.910377676977422},policy_reward_mean={'policy_0': 27.046792012152356, 'policy_1': 27.071242290833645},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3558622236460053, 'mean_raw_obs_processing_ms': 0.44937971651098163, 'mean_inference_ms': 4.643043900569673, 'mean_action_processing_ms': 0.1885209045968464},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 2149.444, 'sample_throughput': 1860.946, 'learn_time_ms': 20909.811, 'learn_throughput': 191.298, 'update_time_ms': 2.795},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.30000000000000004, 'cur_lr': 0.15, 'total_loss': inf, 'policy_loss': 0.27

Trial PPO_diffdemand_fea7d_00000 reported episode_reward_max=57.167392650101775,episode_reward_min=51.12231804601898,episode_reward_mean=53.98373507607108,episode_len_mean=100.0,episodes_this_iter=40,policy_reward_min={'policy_0': 24.185918819608695, 'policy_1': 24.568367919442},policy_reward_max={'policy_0': 29.85594849525549, 'policy_1': 29.314852406282967},policy_reward_mean={'policy_0': 26.791111185732134, 'policy_1': 27.19262389033894},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3132450586248297, 'mean_raw_obs_processing_ms': 0.3957716956568019, 'mean_inference_ms': 4.084008037101896, 'mean_action_processing_ms': 0.16586263376742727},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 1834.716, 'sample_throughput': 2180.174, 'learn_time_ms': 20156.542, 'learn_throughput': 198.447, 'update_time_ms': 2.773},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.45000000000000007, 'cur_lr': 0.15, 'total_loss': inf, 'policy_loss': 0.2

Trial PPO_diffdemand_fea7d_00000 reported episode_reward_max=57.167392650101775,episode_reward_min=50.66014037770275,episode_reward_mean=53.77219008659806,episode_len_mean=100.0,episodes_this_iter=40,policy_reward_min={'policy_0': 24.185918819608695, 'policy_1': 24.568367919442},policy_reward_max={'policy_0': 29.85594849525549, 'policy_1': 29.314852406282967},policy_reward_mean={'policy_0': 26.445710848328396, 'policy_1': 27.32647923826966},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.26946607552876506, 'mean_raw_obs_processing_ms': 0.3422325056800423, 'mean_inference_ms': 3.5115468373732983, 'mean_action_processing_ms': 0.14234878426407493},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 1670.118, 'sample_throughput': 2395.04, 'learn_time_ms': 20644.92, 'learn_throughput': 193.752, 'update_time_ms': 2.763},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.675, 'cur_lr': 0.15, 'total_loss': 1.7478932263329625, 'policy_loss': 0.

Trial PPO_diffdemand_fea7d_00000 reported episode_reward_max=57.167392650101775,episode_reward_min=50.66014037770275,episode_reward_mean=53.4878002718237,episode_len_mean=100.0,episodes_this_iter=40,policy_reward_min={'policy_0': 21.776290429158294, 'policy_1': 24.573441270717304},policy_reward_max={'policy_0': 29.85594849525549, 'policy_1': 29.81121800498625},policy_reward_mean={'policy_0': 26.05488076095648, 'policy_1': 27.432919510867226},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.2497503433265588, 'mean_raw_obs_processing_ms': 0.31911540247014486, 'mean_inference_ms': 3.2857669847915245, 'mean_action_processing_ms': 0.13176612192065573},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 1670.963, 'sample_throughput': 2393.83, 'learn_time_ms': 22117.438, 'learn_throughput': 180.853, 'update_time_ms': 3.046},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.3375, 'cur_lr': 0.15, 'total_loss': 2.1918684523552656, 'policy_loss':

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_diffdemand_fea7d_00000,TERMINATED,,5,118.989,20000,53.4878,57.1674,50.6601,100


In [12]:
#Mixed:  SAC, IMPALA

algo_list=["SAC", "IMPALA"]
for i in range(len(algo_list)):
    exp_name = exp_label + algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )

py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92762) 2021-04-13 13:19:02,727	WARNING sac_tf_policy.py:65 -- When not using a state-preprocessor with SAC, `fcnet_hiddens` will be set to an empty list! Any hidden layer sizes are defined via `policy_model.fcnet_hiddens` and `Q_model.fcnet_hiddens`.
(pid=92762) 2021-04-13 13:19:02,728	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92762) 2021-04-13 13:19:02,746	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=92739) 2021-04-13 13:19:02,759	WARNING sac_tf_policy.py:65 -- When not using a state-preprocessor with SAC, `fcnet_hiddens` will be set to an empty list! Any hidden layer sizes are defined via `policy_model.

Trial SAC_diffdemand_5155b_00000 reported episode_reward_max=55.575183122390555,episode_reward_min=52.391939874876066,episode_reward_mean=54.27257206398218,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.53375788544306, 'policy_1': 25.913256888262424},policy_reward_max={'policy_0': 28.671747207665536, 'policy_1': 30.1309050083817},policy_reward_mean={'policy_0': 26.864750615277046, 'policy_1': 27.40782144870513},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3304556222494282, 'mean_raw_obs_processing_ms': 1.0601142151607807, 'mean_inference_ms': 4.556040440079769, 'mean_action_processing_ms': 0.18193167038056468},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 34.315, 'learn_throughput': 7460.214, 'update_time_ms': 4.548},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.09901857, 0.0873909 , 0.1596272 , 0.19045389, 0.09468174,
       0.08579075, 0.10069013, 0.15731788, 0.05932772, 0.04298234,
   

Trial SAC_diffdemand_5155b_00000 reported episode_reward_max=55.575183122390555,episode_reward_min=52.391939874876066,episode_reward_mean=54.13910045717052,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.53375788544306, 'policy_1': 25.913256888262424},policy_reward_max={'policy_0': 28.671747207665536, 'policy_1': 30.1309050083817},policy_reward_mean={'policy_0': 26.72876307471733, 'policy_1': 27.41033738245317},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.32533209082356496, 'mean_raw_obs_processing_ms': 1.0403998982103047, 'mean_inference_ms': 4.380468241436147, 'mean_action_processing_ms': 0.17952118989258106},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 31.333, 'learn_throughput': 8170.388, 'update_time_ms': 4.082},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.02505338, 0.03147662, 0.00974441, 0.10991204, 0.03075695,
       0.08248162, 0.07839036, 0.05248988, 0.2226845 , 0.07239628,
   

Trial SAC_diffdemand_5155b_00000 reported episode_reward_max=55.665607886747594,episode_reward_min=52.391939874876066,episode_reward_mean=54.14437523427089,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.53375788544306, 'policy_1': 24.027579683337688},policy_reward_max={'policy_0': 29.385364331454493, 'policy_1': 30.1309050083817},policy_reward_mean={'policy_0': 26.886472699970263, 'policy_1': 27.257902534300626},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.31992371573860856, 'mean_raw_obs_processing_ms': 1.0209530161329938, 'mean_inference_ms': 4.2403296101059444, 'mean_action_processing_ms': 0.17680026209323874},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 31.387, 'learn_throughput': 8156.263, 'update_time_ms': 4.359},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.13914514, 0.11717105, 0.05945349, 0.02869225, 0.03305221,
       0.06176639, 0.06476355, 0.08635569, 0.07463455, 0.06279731,


Trial SAC_diffdemand_5155b_00000 reported episode_reward_max=55.665607886747594,episode_reward_min=52.391939874876066,episode_reward_mean=54.08506074336616,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 24.53375788544306, 'policy_1': 24.027579683337688},policy_reward_max={'policy_0': 29.385364331454493, 'policy_1': 30.1309050083817},policy_reward_mean={'policy_0': 26.912889870164534, 'policy_1': 27.172170873201615},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.3151777768213432, 'mean_raw_obs_processing_ms': 1.0045095755193376, 'mean_inference_ms': 4.127350477016877, 'mean_action_processing_ms': 0.1743458938670615},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 29.845, 'learn_throughput': 8577.555, 'update_time_ms': 4.045},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.02538395, 0.10494709, 0.05487061, 0.03742647, 0.03369951,
       0.0434525 , 0.04732823, 0.05226421, 0.03351331, 0.04197621,
   

...

Trial SAC_diffdemand_5155b_00000 reported episode_reward_max=56.16971294857838,episode_reward_min=51.96061778905331,episode_reward_mean=54.00863748182219,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 23.635681222981923, 'policy_1': 24.14346105532576},policy_reward_max={'policy_0': 28.97552966769836, 'policy_1': 30.785241210711774},policy_reward_mean={'policy_0': 26.997462009378136, 'policy_1': 27.01117547244405},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.2932574772443507, 'mean_raw_obs_processing_ms': 0.9428099666445194, 'mean_inference_ms': 3.4814020150498544, 'mean_action_processing_ms': 0.1621193014614093},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 28.771, 'learn_throughput': 8897.857, 'update_time_ms': 3.54},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.20400143, 0.02300262, 0.02227783, 0.0663929 , 0.08792162,
       0.0708456 , 0.02066755, 0.16983843, 0.10351276, 0.12308359,
     

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
SAC_diffdemand_5155b_00000,TERMINATED,,20,176.474,20500,54.0086,56.1697,51.9606,100


ported in the long term
(pid=93050) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=93050) Instructions for updating:
(pid=93050) non-resource variables are not supported in the long term
(pid=93050) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=93050) Instructions for updating:
(pid=93050) non-resource variables are not supported in the long term
(pid=93051) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/

Trial IMPALA_diffdemand_c30a2_00000 reported episode_reward_max=56.95225260440203,episode_reward_min=51.341387656588275,episode_reward_mean=54.18095969886395,episode_len_mean=100.0,episodes_this_iter=269,policy_reward_min={'policy_0': 24.625006323530005, 'policy_1': 23.675805086061146},policy_reward_max={'policy_0': 30.02554749121364, 'policy_1': 29.00262820821685},policy_reward_mean={'policy_0': 27.168781767690827, 'policy_1': 27.012177931173117},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.2770818503895667, 'mean_raw_obs_processing_ms': 0.3508450224630989, 'mean_inference_ms': 2.873557598560234, 'mean_action_processing_ms': 0.14571034711722666},off_policy_estimator={},num_healthy_workers=10,timers={'sample_time_ms': 196.99, 'sample_throughput': 2538.198},info={'num_steps_sampled': 28050, 'num_steps_trained': 27500, 'num_weight_broadcasts': 52, 'learner_queue': {'size_count': 56, 'size_mean': 0.06, 'size_std': 0.23748684174075835, 'size_quantiles': [0.0, 0.0, 0.0, 0.0, 1.0]},

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
IMPALA_diffdemand_c30a2_00000,TERMINATED,,2,10.8159,28050,54.181,56.9523,51.3414,100


### Continuous Space


In [13]:
#DDGP uses its own exploration config
# See exploration config in https://github.com/ray-project/ray/blob/master/rllib/utils/exploration/ornstein_uhlenbeck_noise.pyDDPG 
exploration_config_cont = {
        # DDPG uses OrnsteinUhlenbeck (stateful) noise to be added to NN-output
        # actions (after a possible pure random phase of n timesteps).
        "type": "OrnsteinUhlenbeckNoise",
        "final_scale": 0.02,
        "scale_timesteps": 100000,
    }

training_config_cont=training_config.copy()
env_config_cont=env_config.copy()
training_config_cont["exploration_config"] = exploration_config_cont
env_config_cont["mkt_config"]["space_type"] = "Continuous"

env=DiffDemand(env_config_cont)
training_config_cont["env_config"] = env_config_cont
training_config_cont["multiagent"]["policies"] =  {
            policy_ids[i]: (None, env.observation_space[f"agent_{i}"], env.action_space[f"agent_{i}"],{},) for i in range(env.n_agents)
}
#print(env_config)
print(training_config_cont)
print(env.action_space)


{'gamma': 0.95, 'lr': 0.15, 'env': 'diffdemand', 'exploration_config': {'type': 'OrnsteinUhlenbeckNoise', 'final_scale': 0.02, 'scale_timesteps': 100000}, 'env_config': {'mkt_config': {'lower_price': [1.3699999999999999, 1.3699999999999999], 'higher_price': [2.03, 2.03], 'parameteres': {'cost': [1, 1], 'values': [2, 2], 'ext_demand': 0, 'substitution': 0.25}, 'space_type': 'Continuous', 'gridpoints': 16}}, 'horizon': 100, 'soft_horizon': True, 'no_done_at_end': True, 'multiagent': {'policies': {'policy_0': (None, Box(1.3699999999999999, 2.03, (2,), float64), Box(1.3699999999999999, 2.03, (1,), float64), {}), 'policy_1': (None, Box(1.3699999999999999, 2.03, (2,), float64), Box(1.3699999999999999, 2.03, (1,), float64), {})}, 'policy_mapping_fn': <function <lambda> at 0x1a092b550>}, 'framework': 'torch', 'num_workers': 10, 'num_gpus': 0, 'timesteps_per_iteration': 1000, 'normalize_actions': False}
{'agent_0': Box(1.3699999999999999, 2.03, (1,), float64), 'agent_1': Box(1.3699999999999999,

In [16]:
#COntinuous action space DQN

algo_list=["DDPG", "TD3", "SAC"]
for i in range(len(algo_list)):
    exp_name = exp_label + "_cont_" + algo_list[i]
    results = tune.run(
        algo_list[i],
        name=exp_name,
        config=training_config_cont,
        #checkpoint_freq=250,
        checkpoint_at_end=True,
        stop=stop,
        callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
        verbose=verbosity
    )

using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
(pid=93503)   torch.from_numpy(self.action_space.low).float())
(pid=93503) 2021-04-13 13:28:47,139	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=93503) 2021-04-13 13:28:47,175	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=93503) 2021-04-13 13:28:47,178	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=93503) 2021-04-13 13:28:47,135	WARNING deprecation.py:33 -- DeprecationWarning: `

Trial DDPG_diffdemand_b060d_00000 reported episode_reward_max=60.491620774571025,episode_reward_min=52.46684292109934,episode_reward_mean=57.326790616273605,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 26.339421651271653, 'policy_1': 25.35429049001401},policy_reward_max={'policy_0': 30.260832257976894, 'policy_1': 31.680466056709182},policy_reward_mean={'policy_0': 28.25202278478015, 'policy_1': 29.074767831493496},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.1675674492021824, 'mean_raw_obs_processing_ms': 0.4473235244747957, 'mean_inference_ms': 1.8842535507571732, 'mean_action_processing_ms': 0.16167683134745528},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 23.923, 'learn_throughput': 10701.184, 'update_time_ms': 2.44},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.32419735193252563, 'critic_loss': 0.0006890440708957613, 'mean_q': 0.33072876930236816, 'max_q': 0.3812454342842102, 'min_q': 0.

Trial DDPG_diffdemand_b060d_00000 reported episode_reward_max=60.491620774571025,episode_reward_min=48.0637188349328,episode_reward_mean=54.4413100569494,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 14.469945365393617, 'policy_1': 25.35429049001401},policy_reward_max={'policy_0': 30.260832257976894, 'policy_1': 34.479549199199774},policy_reward_mean={'policy_0': 23.735443997803532, 'policy_1': 30.705866059145887},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.16687435604927642, 'mean_raw_obs_processing_ms': 0.4449957377765957, 'mean_inference_ms': 1.8652855971551052, 'mean_action_processing_ms': 0.16043659172629268},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 23.668, 'learn_throughput': 10816.443, 'update_time_ms': 2.372},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.346090704202652, 'critic_loss': 0.0004901136853732169, 'mean_q': 0.35927969217300415, 'max_q': 0.4919116795063019, 'min_q': 0.21

Trial DDPG_diffdemand_b060d_00000 reported episode_reward_max=60.491620774571025,episode_reward_min=37.76736021593465,episode_reward_mean=50.39035308466814,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 12.025090484603604, 'policy_1': 25.35429049001401},policy_reward_max={'policy_0': 30.260832257976894, 'policy_1': 34.479549199199774},policy_reward_mean={'policy_0': 20.870171775190816, 'policy_1': 29.520181309477344},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.16708133055317237, 'mean_raw_obs_processing_ms': 0.44508611233124834, 'mean_inference_ms': 1.8613869441144264, 'mean_action_processing_ms': 0.16030982981071937},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 23.716, 'learn_throughput': 10794.293, 'update_time_ms': 2.375},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.4291670024394989, 'critic_loss': 0.0005613394896499813, 'mean_q': 0.39633363485336304, 'max_q': 0.5289214253425598, 'min_q': 

Trial DDPG_diffdemand_b060d_00000 reported episode_reward_max=60.491620774571025,episode_reward_min=36.47422436342299,episode_reward_mean=47.6842523234848,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 12.025090484603604, 'policy_1': 18.394771361195616},policy_reward_max={'policy_0': 30.260832257976894, 'policy_1': 34.479549199199774},policy_reward_mean={'policy_0': 20.3548167634982, 'policy_1': 27.329435559986614},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.1670234583522484, 'mean_raw_obs_processing_ms': 0.44434684337116365, 'mean_inference_ms': 1.8562907568871176, 'mean_action_processing_ms': 0.1599823373245354},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 23.562, 'learn_throughput': 10865.061, 'update_time_ms': 2.343},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'actor_loss': -0.4752320945262909, 'critic_loss': 0.0005116881220601499, 'mean_q': 0.4554104804992676, 'max_q': 0.5822457075119019, 'min_q': 0.281

...

Trial SAC_diffdemand_5051f_00000 reported episode_reward_max=35.56906399865392,episode_reward_min=35.56906399865392,episode_reward_mean=35.56906399865391,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_max={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_mean={'policy_0': 17.784531999326926, 'policy_1': 17.784531999326926},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.22712097581152502, 'mean_raw_obs_processing_ms': 0.5923059751214672, 'mean_inference_ms': 2.7338986330391135, 'mean_action_processing_ms': 0.21781893369601973},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 35.592, 'learn_throughput': 7192.534, 'update_time_ms': 3.452},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([1.24042392e-01, 2.36801624e-01, 1.40071201e+00, 2.84018755e-01,
       8.44697833e-01, 1.89004779e-01, 2.05270052e-01, 8.64665389e

Trial SAC_diffdemand_5051f_00000 reported episode_reward_max=35.56906399865392,episode_reward_min=35.56906399865392,episode_reward_mean=35.56906399865391,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_max={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_mean={'policy_0': 17.784531999326926, 'policy_1': 17.784531999326926},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.22635807901247385, 'mean_raw_obs_processing_ms': 0.5911363832935236, 'mean_inference_ms': 2.7193182620235046, 'mean_action_processing_ms': 0.21698726162177556},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 35.815, 'learn_throughput': 7147.791, 'update_time_ms': 3.447},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.29183316, 0.17442852, 0.2975341 , 0.27172184, 0.22970313,
       0.07796544, 0.05606085, 0.29716033, 0.18060118, 0.13514036,
   

Trial SAC_diffdemand_5051f_00000 reported episode_reward_max=35.56906399865392,episode_reward_min=35.56906399865392,episode_reward_mean=35.56906399865391,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_max={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_mean={'policy_0': 17.784531999326926, 'policy_1': 17.784531999326926},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.22561915724990264, 'mean_raw_obs_processing_ms': 0.5900174837469726, 'mean_inference_ms': 2.705746179532428, 'mean_action_processing_ms': 0.21616237390417267},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 35.333, 'learn_throughput': 7245.378, 'update_time_ms': 3.435},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.1264171 , 0.29616696, 0.27095938, 0.07411581, 0.0102213 ,
       0.98772615, 0.6187932 , 0.2176131 , 0.29097664, 2.0123353 ,
    

Trial SAC_diffdemand_5051f_00000 reported episode_reward_max=35.56906399865392,episode_reward_min=35.56906399865392,episode_reward_mean=35.56906399865391,episode_len_mean=100.0,episodes_this_iter=10,policy_reward_min={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_max={'policy_0': 17.78453199932693, 'policy_1': 17.78453199932693},policy_reward_mean={'policy_0': 17.784531999326926, 'policy_1': 17.784531999326926},custom_metrics={},sampler_perf={'mean_env_wait_ms': 0.22481390682942082, 'mean_raw_obs_processing_ms': 0.5885994272560333, 'mean_inference_ms': 2.692354194238218, 'mean_action_processing_ms': 0.21530319862692118},off_policy_estimator={},num_healthy_workers=10,timers={'learn_time_ms': 35.198, 'learn_throughput': 7273.225, 'update_time_ms': 3.401},info={'learner': {'policy_0': {'allreduce_latency': 0.0, 'td_error': array([0.32231015, 0.331572  , 0.04131418, 0.1415388 , 0.29455274,
       0.330593  , 0.05168229, 0.19234914, 0.05784518, 0.21074766,
    

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
SAC_diffdemand_5051f_00000,TERMINATED,,20,146.848,20500,35.5691,35.5691,35.5691,100


In [ ]:
shutdown()